In [ ]:
import pandas as pd

#cargar todas las matrices de distancias de 40 nodos
ruta1 = pd.read_excel('Ruta1_40nodos.xlsx', header =None)
ruta2 = pd.read_excel('Ruta2_40nodos.xlsx', header =None)
ruta3 = pd.read_excel('Ruta3_40nodos.xlsx', header =None)
ruta4 = pd.read_excel('Ruta4_40nodos.xlsx', header =None)
ruta5 = pd.read_excel('Ruta5_40nodos.xlsx', header =None)
ruta6 = pd.read_excel('Ruta6_40nodos.xlsx', header =None)
ruta7 = pd.read_excel('Ruta7_40nodos.xlsx', header =None)
ruta8 = pd.read_excel('Ruta8_40nodos.xlsx', header =None)
ruta9 = pd.read_excel('Ruta9_40nodos.xlsx', header =None)
ruta10 = pd.read_excel('Ruta10_40nodos.xlsx', header =None)

ruta1.head()

,0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0,999.000000,3.735415,3.832239,6.045863,4.313529,8.207288,8.125308,3.295205,4.243462,6.416005,...,7.575399,5.537093,5.235365,6.953542,7.476871,6.005915,5.632561,7.954472,10.057465,2.179891
1,3.735415,999.000000,4.699746,8.902923,7.746917,11.936186,11.848213,6.523024,7.641003,10.111892,...,11.283378,1.864715,3.305986,10.590905,11.208109,9.701944,1.912498,11.628216,13.145535,2.824182
2,3.832239,4.699746,999.000000,4.817080,4.783992,9.593977,9.428827,6.837916,4.626545,7.629016,...,9.672476,6.328416,3.226974,7.758099,8.969536,8.327963,5.859293,8.865480,9.017650,2.164096
3,6.045863,8.902923,4.817080,999.000000,2.409944,5.953283,5.731248,7.102020,2.338749,4.273126,...,6.832761,10.726132,8.040885,3.915677,5.563976,6.122073,10.431243,4.884053,4.263516,6.088405
4,4.313529,7.746917,4.783992,2.409944,999.000000,4.812162,4.644880,4.726051,0.162419,2.850714,...,5.096637,9.611235,7.816660,3.015048,4.206310,4.034738,9.504336,4.119903,5.821728,5.148402


In [ ]:
import pandas as pd
import numpy as np
import random

df = pd.read_excel('Ruta2_40nodos.xlsx', header =None)

distancias = np.array(df.values)
column_values = df.columns.values

tamano_pob = len(column_values)
mut_prob = 0.01
generaciones = 10000
equivalencias = {i: df.columns.values[i] for i in range(len(df.columns.values))}
punto_cruce = random.randint(0, tamano_pob-1)

# Crear población inicial
def poblacion_inicial(tamano_pob):
    poblacion = []
    for _ in range(tamano_pob):
        ruta = [0]  # Siempre comienza con el punto 0
        disponibles = list(range(1, tamano_pob))
        while len(ruta) < tamano_pob:
            valor = random.choice(disponibles)
            ruta.append(valor)
            disponibles.remove(valor)
        poblacion.append(ruta)
    return poblacion

def fitness(ruta):
    total = 0
    new_ruta = [equivalencias[r] for r in ruta]
    total = sum([df.loc[int(new_ruta[i]), new_ruta[i+1]] for i in range(len(new_ruta) - 1)])
    return 1 / total

# Escogiendo el mejor cromosoma
def mejor(poblacion):
    fitness_val = [fitness(p) for p in poblacion]

    maximo = max(fitness_val)
    max_index = fitness_val.index(maximo)
    return poblacion[max_index]

# Cruce
def cruce(p1, p2):
    n = len(p1)
    start = random.randint(0, n - 1)
    end = random.randint(start, n - 1)
    child1 = [-1] * n
    child2 = [-1] * n

    # Copia el rango del primer padre a los hijos
    for i in range(start, end + 1):
        child1[i] = p1[i]
        child2[i] = p2[i]

    # Completa los hijos con genes del otro padre sin duplicados
    j1 = 0
    j2 = 0
    for i in range(n):
        if child1[i] == -1:
            while p2[j1] in child1:
                j1 += 1
            child1[i] = p2[j1]
        if child2[i] == -1:
            while p1[j2] in child2:
                j2 += 1
            child2[i] = p1[j2]

    return child1, child2


def algoritmo_genetico(distancias, tamano_pob, mut_prob, generaciones):
    poblacion = poblacion_inicial(tamano_pob)
    for _ in range(generaciones):
        valores_fitness = [fitness(p) for p in poblacion]


        padres = random.choices(poblacion, weights=valores_fitness, k=2)

        hijo1, hijo2 = cruce(padres[0], padres[1])

        if random.uniform(0,1) < mut_prob:
            muts = random.sample(range(40), 2)
            punto_mut1 = muts[0]
            punto_mut2 = muts[1]
            hijo1[punto_mut1], hijo1[punto_mut2] = hijo1[punto_mut2], hijo1[punto_mut1]
        if random.uniform(0,1) < mut_prob:
            muts = random.sample(range(40), 2)
            punto_mut1 = muts[0]
            punto_mut2 = muts[1]
            hijo2[punto_mut1], hijo2[punto_mut2] = hijo2[punto_mut2], hijo2[punto_mut1]

        poblacion = [hijo1, hijo2] + poblacion
        poblacion.sort(key=lambda x: fitness(x), reverse=True)
        poblacion = poblacion[:tamano_pob]


    ruta = mejor(poblacion)
    ruta.append(ruta[0])
    new_ruta = [equivalencias[r] for r in ruta]
    distancia = sum([df.loc[int(new_ruta[i]), new_ruta[i+1]] for i in range(len(new_ruta) - 1)])
    return new_ruta, distancia


ruta, distancia = algoritmo_genetico(distancias, tamano_pob, mut_prob, generaciones)
print("\nParametros:")
print("Tamaño de Población:", tamano_pob)
print("Probabilidad de Mutación:", mut_prob)
print("Punto de Cruce:", punto_cruce)
print("Generaciones:", generaciones, "\n")
print("Mejor Ruta:", ruta)
print("Menor Distancia:", distancia)


Parametros:
Tamaño de Población: 40
Probabilidad de Mutación: 0.01
Punto de Cruce: 23
Generaciones: 10000 

Mejor Ruta: [0, 19, 31, 30, 1, 12, 37, 35, 22, 36, 25, 26, 34, 4, 39, 33, 2, 20, 27, 9, 38, 8, 6, 28, 21, 32, 10, 15, 13, 18, 11, 5, 16, 29, 3, 23, 7, 24, 14, 17, 0]
Menor Distancia: 109.47943086524874


In [ ]:
import pandas as pd
import numpy as np
import random

df = pd.read_excel('Ruta3_40nodos.xlsx', header =None)

distancias = np.array(df.values)
column_values = df.columns.values

tamano_pob = len(column_values)
mut_prob = 0.01
generaciones = 10000
equivalencias = {i: df.columns.values[i] for i in range(len(df.columns.values))}
punto_cruce = random.randint(0, tamano_pob-1)

# Crear población inicial
def poblacion_inicial(tamano_pob):
    poblacion = []
    for _ in range(tamano_pob):
        ruta = [0]  # Siempre comienza con el punto 0
        disponibles = list(range(1, tamano_pob))
        while len(ruta) < tamano_pob:
            valor = random.choice(disponibles)
            ruta.append(valor)
            disponibles.remove(valor)
        poblacion.append(ruta)
    return poblacion

def fitness(ruta):
    total = 0
    new_ruta = [equivalencias[r] for r in ruta]
    total = sum([df.loc[int(new_ruta[i]), new_ruta[i+1]] for i in range(len(new_ruta) - 1)])
    return 1 / total

# Escogiendo el mejor cromosoma
def mejor(poblacion):
    fitness_val = [fitness(p) for p in poblacion]

    maximo = max(fitness_val)
    max_index = fitness_val.index(maximo)
    return poblacion[max_index]

# Cruce
def cruce(p1, p2):
    n = len(p1)
    start = random.randint(0, n - 1)
    end = random.randint(start, n - 1)
    child1 = [-1] * n
    child2 = [-1] * n

    # Copia el rango del primer padre a los hijos
    for i in range(start, end + 1):
        child1[i] = p1[i]
        child2[i] = p2[i]

    # Completa los hijos con genes del otro padre sin duplicados
    j1 = 0
    j2 = 0
    for i in range(n):
        if child1[i] == -1:
            while p2[j1] in child1:
                j1 += 1
            child1[i] = p2[j1]
        if child2[i] == -1:
            while p1[j2] in child2:
                j2 += 1
            child2[i] = p1[j2]

    return child1, child2


def algoritmo_genetico(distancias, tamano_pob, mut_prob, generaciones):
    poblacion = poblacion_inicial(tamano_pob)
    for _ in range(generaciones):
        valores_fitness = [fitness(p) for p in poblacion]


        padres = random.choices(poblacion, weights=valores_fitness, k=2)

        hijo1, hijo2 = cruce(padres[0], padres[1])

        if random.uniform(0,1) < mut_prob:
            muts = random.sample(range(40), 2)
            punto_mut1 = muts[0]
            punto_mut2 = muts[1]
            hijo1[punto_mut1], hijo1[punto_mut2] = hijo1[punto_mut2], hijo1[punto_mut1]
        if random.uniform(0,1) < mut_prob:
            muts = random.sample(range(40), 2)
            punto_mut1 = muts[0]
            punto_mut2 = muts[1]
            hijo2[punto_mut1], hijo2[punto_mut2] = hijo2[punto_mut2], hijo2[punto_mut1]

        poblacion = [hijo1, hijo2] + poblacion
        poblacion.sort(key=lambda x: fitness(x), reverse=True)
        poblacion = poblacion[:tamano_pob]


    ruta = mejor(poblacion)
    ruta.append(ruta[0])
    new_ruta = [equivalencias[r] for r in ruta]
    distancia = sum([df.loc[int(new_ruta[i]), new_ruta[i+1]] for i in range(len(new_ruta) - 1)])
    return new_ruta, distancia


ruta, distancia = algoritmo_genetico(distancias, tamano_pob, mut_prob, generaciones)
print("\nParametros:")
print("Tamaño de Población:", tamano_pob)
print("Probabilidad de Mutación:", mut_prob)
print("Punto de Cruce:", punto_cruce)
print("Generaciones:", generaciones, "\n")
print("Mejor Ruta:", ruta)
print("Menor Distancia:", distancia)


Parametros:
Tamaño de Población: 40
Probabilidad de Mutación: 0.01
Punto de Cruce: 22
Generaciones: 10000 

Mejor Ruta: [0, 1, 4, 28, 18, 29, 24, 16, 15, 35, 14, 8, 23, 38, 6, 9, 17, 3, 13, 22, 27, 10, 26, 7, 5, 19, 25, 33, 31, 34, 20, 11, 2, 30, 39, 36, 12, 32, 21, 37, 0]
Menor Distancia: 113.98800422230977
